# Criação do modelo de Machine Learning, capaz de fazer a verificação dos sentimentos de cada frase

In [1]:
# Imports
import numpy as np
import pandas as pd
from unidecode import unidecode
import spacy

import nltk
nltk.download('stopwords')
nltk.download('rslp')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alyss\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\alyss\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

In [13]:
!pip install scikit-learn==1.6.0

     --------------------------------------- 11.1/11.1 MB 10.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.1
    Uninstalling scikit-learn-1.2.1:
      Successfully uninstalled scikit-learn-1.2.1


ERROR: Could not install packages due to an OSError: [WinError 5] Acesso negado: 'C:\\Users\\alyss\\anaconda3\\Lib\\site-packages\\~klearn\\.libs\\msvcp140.dll'
Consider using the `--user` option or check the permissions.



In [14]:
pip show scikit-learn

Name: scikit-learn
Version: 1.6.0
Summary: A set of python modules for machine learning and data mining
Home-page: 
Author: 
Author-email: 
License: BSD 3-Clause License
         
         Copyright (c) 2007-2024 The scikit-learn developers.
         All rights reserved.
         
         Redistribution and use in source and binary forms, with or without
         modification, are permitted provided that the following conditions are met:
         
         * Redistributions of source code must retain the above copyright notice, this
           list of conditions and the following disclaimer.
         
         * Redistributions in binary form must reproduce the above copyright notice,
           this list of conditions and the following disclaimer in the documentation
           and/or other materials provided with the distribution.
         
         * Neither the name of the copyright holder nor the names of its
           contributors may be used to endorse or promote products deri

In [15]:
data = pd.read_csv("C:/Projetos Pessoais/dados/imdb-reviews-pt-br.csv")

In [16]:
data.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [17]:
# Info do conjunto de dados
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49459 entries, 0 to 49458
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         49459 non-null  int64 
 1   text_en    49459 non-null  object
 2   text_pt    49459 non-null  object
 3   sentiment  49459 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.5+ MB


In [18]:
data['sentiment_int'] = data['sentiment'].map({"pos":1, "neg":0})

In [19]:
data.sample()

,id,text_en,text_pt,sentiment,sentiment_int
353,354,I watched 5% of this movie tonight and you may...,Eu assisti 5% deste filme hoje à noite e você ...,neg,0


In [11]:
!python -m spacy download pt_core_news_lg

     ------------------------------------ 568.2/568.2 MB 678.9 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [20]:
# Load dados em portugues

nlp = spacy.load("pt_core_news_lg")

### <font color="red"> Criando função de Lematização para os verbos

In [21]:
def lematizer(text):

   # Lista de sentenças e documentos
   sentencas = []
   doc = nlp(text) # leitura do text e interpretar em lg de processamento

   for palavra in doc:
      if palavra.pos_ == "VERB":
         sentencas.append(palavra.lemma_) # se for verbo, extrai o lemma
      else:
         sentencas.append(palavra.orth_) # se n for verbo, extrai a palavra

   return " ".join(sentencas)

In [22]:
data['text'] = data['text_pt'].apply(str.lower)
data['text'] = data['text'].apply(unidecode)
data['text'] = data['text'].apply(lematizer)

In [50]:
data.text

0        mais uma vez , o sr. costner arrumar um filme ...
1        este e um exemplo do motivo pelo qual a maiori...
2        primeiro de tudo eu odeior esses raps imbecis ...
3        nem mesmo os beatles puder escrever musicas qu...
4        filmes de fotos de latao nao e uma palavra apr...
                               ...                        
49454    como a media de votos era muito baixa , e o fa...
49455    o enredo ter algumas reviravoltas infelizes e ...
49456    estou espantado com a forma como este filme e ...
49457    a christmas together realmente vir antes do me...
49458    o drama romantico da classe trabalhadora do di...
Name: text, Length: 49459, dtype: object

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49459 entries, 0 to 49458
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             49459 non-null  int64 
 1   text_en        49459 non-null  object
 2   text_pt        49459 non-null  object
 3   sentiment      49459 non-null  object
 4   sentiment_int  49459 non-null  int64 
 5   text           49459 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.3+ MB


In [24]:
# Salvando novo DF processado - Endpoint
data.to_csv("C:/Projetos Pessoais/dados/imdb_dados_processados_sklearn1_6_0.csv")

In [25]:
data = pd.read_csv("C:/Projetos Pessoais/dados/imdb_dados_processados_sklearn1_6_0.csv")

In [26]:
df = data.copy()

### <font color="red"> Tratamento de Stopwords

In [27]:
stop_words = nltk.corpus.stopwords.words('portuguese')

In [29]:
stop_words[:10]

['a',
 'à',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as']

### <font color="red"> Contagem de Unigramas

In [30]:
vect_uni_cv = CountVectorizer(ngram_range=(1,1), stop_words = stop_words)
vect_uni_cv.fit(df.text)
text_vect_uni_cv = vect_uni_cv.transform(df.text)

In [51]:
import pickle

with open("./api_flask/models/vectorizer.pkl", "wb") as f:
    pickle.dump(vect_uni_cv, f)

In [35]:
x_train, x_test, y_train, y_test = train_test_split(text_vect_uni_cv,
                                                    df["sentiment_int"],
                                                    test_size = 0.2,
                                                    random_state = 123)

In [36]:
x_train

<39567x115329 sparse matrix of type '<class 'numpy.int64'>'
	with 4081641 stored elements in Compressed Sparse Row format>

### <font color="red"> Modelo v1 => Regressão Logística

In [37]:
# Logistic Regression

lr = LogisticRegression()
lr.fit(x_train, y_train)

LogisticRegression()

In [38]:
lr_prev_modelo_1 = lr.predict(x_test)

# Acurácia
acc_v1 = accuracy_score(lr_prev_modelo_1, y_test)
acc_v1

0.8743429033562474

In [40]:
import pickle

modelo_v1_metricas = {
 "Modelo": "Logistic Regression",
 "Versão": "1",
 "Detalhes": " - ",
 "F1_Score": round(f1_score(lr_prev_modelo_1, y_test), 3),
 "Acurácia": round(accuracy_score(lr_prev_modelo_1, y_test), 3),
 "AUC": round(roc_auc_score(y_test, lr_prev_modelo_1), 3) #AUC inverte a ordem dos dados -  primeiro real depois previsões
}

with open("./api_flask/models/modelo_v1_LR.pkl", "wb") as file:  # "wb" significa escrita em modo binário
    pickle.dump(lr_prev_modelo_1, file)

modelo_v1_metricas

{'Modelo': 'Logistic Regression',
 'Versão': '1',
 'Detalhes': ' - ',
 'F1_Score': 0.877,
 'Acurácia': 0.874,
 'AUC': 0.874}

### <font color="red">Modelo v2 => Regressão Logística com Grid Search

In [41]:
lr_grid = LogisticRegression()
lr_grid_values = {
   'penalty': ['l1','l2'],
   'C': [0.001, 0.009, 0.01, .09, 1, 5, 10, 25]
}

In [42]:
lr_grid_cv = GridSearchCV(lr_grid,
                          param_grid = lr_grid_values,
                          scoring = 'accuracy',
                          cv=5)

In [43]:
lr_grid_cv.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.009, 0.09, 1, 5, 10, 25],
                         'penalty': ['l1', 'l2']},
             scoring='accuracy')

In [44]:
lr_grid_cv.best_estimator_

LogisticRegression(C=0.09)

In [45]:
y_predict_lr_GS = lr_grid_cv.predict(x_test)

acc_v2 = accuracy_score(y_predict_lr_GS, y_test)
acc_v2

0.8858673675697534

In [46]:
modelo_v2_metricas = {
 "Modelo": "Logistic Regression GS",
 "Versão": "2",
 "Detalhes": "C=09",
 "F1_Score": round(f1_score(y_predict_lr_GS, y_test), 3),
 "Acurácia": round(accuracy_score(y_predict_lr_GS, y_test), 3),
 "AUC": round(roc_auc_score(y_test, y_predict_lr_GS), 3) #AUC inverte a ordem dos dados -  primeiro real depois previsões
}

with open("./api_flask/models/modelo_v2_LRGS.pkl", "wb") as file:  # "wb" significa escrita em modo binário
    pickle.dump(lr_grid_cv, file)
modelo_v2_metricas

{'Modelo': 'Logistic Regression GS',
 'Versão': '2',
 'Detalhes': 'C=09',
 'F1_Score': 0.888,
 'Acurácia': 0.886,
 'AUC': 0.886}

In [54]:
import joblib
joblib.dump(y_predict_lr_GS, './app/models/modelo_v2.joblib')

['./app/models/modelo_v2.joblib']

### <font color="red">Modelo v3 => Multinomial Naive Bayes

In [48]:
naive_bayes = MultinomialNB()

param_grid_nb = {
    'alpha': [0.01, 0.001, 0.1, 1, 10],  # Regularização
    'fit_prior': [True, False]     # Se as probabilidades aprioristas devem ser ajustadas
}

naive_Bayes_model_v3 = GridSearchCV(naive_bayes,
                          param_grid = param_grid_nb,
                          cv=5,
                          n_jobs=-1)

naive_Bayes_model_v3.fit(x_train, y_train)

naive_Bayes_model_v3.best_estimator_

exception calling callback for <Future at 0x23d355440d0 state=finished raised BrokenProcessPool>
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "c:\Users\alyss\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py", line 391, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "c:\Users\alyss\anaconda3\lib\multiprocessing\queues.py", line 122, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute '_passthrough_scorer' on <module 'sklearn.metrics._scorer' from 'c:\\Users\\alyss\\anaconda3\\lib\\site-packages\\sklearn\\metrics\\_scorer.py'>
"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\alyss\anaconda3\lib\site-packages\joblib\externals\loky\_base.py", line 26, in _invoke_callbacks
    callback(self)
  File "c:\Users\alyss\anaconda3\lib\site-packages\joblib\parallel.py", line 3

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [28]:
naive_bayes_predict_v3 = naive_Bayes_model_v3.predict(x_test)

In [30]:
modelo_v3_metricas = {
 "Modelo": "Multinomial Naive Bayes",
 "Versão": "3",
 "Detalhes": "alfa=1",
 "F1_Score": round(f1_score(naive_bayes_predict_v3, y_test), 3),
 "Acurácia": round(accuracy_score(naive_bayes_predict_v3, y_test), 3),
 "AUC": round(roc_auc_score(y_test, naive_bayes_predict_v3), 3) #AUC inverte a ordem dos dados -  primeiro real depois previsões
}

with open("./models/modelo_v3.pkl", "wb") as file:  # "wb" significa escrita em modo binário
    pickle.dump(naive_Bayes_model_v3, file)


modelo_v3_metricas

{'Modelo': 'Multinomial Naive Bayes',
 'Versão': '3',
 'Detalhes': 'alfa=1',
 'F1_Score': 0.856,
 'Acurácia': 0.859,
 'AUC': 0.859}

### <font color="red">Modelo v4 => Decision Tree com Grid Search

In [31]:
clf_decision_tree = DecisionTreeClassifier()

params_decision_tree = {
 "criterion": ["gini", "entropy"],
 "max_depth": [3, None],
 "min_samples_leaf": [1, 3, 10],
 "min_samples_split": [2, 3],
 "max_depth": [1,3,10]
}

modelo_v2_decisionTree = GridSearchCV(clf_decision_tree,
                                      params_decision_tree,
                                      scoring='accuracy',
                                      cv = 5)

modelo_v2_decisionTree.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [1, 3, 10],
                         'min_samples_leaf': [1, 3, 10],
                         'min_samples_split': [2, 3]},
             scoring='accuracy')

In [33]:
modelo_v2_decisionTree.best_estimator_

DecisionTreeClassifier(max_depth=10, min_samples_leaf=3)

In [32]:
decisionTree_predict_v4 = modelo_v2_decisionTree.predict(x_test)

In [35]:
modelo_v4_metricas = {
 "Modelo": "Decision Tree",
 "Versão": "4",
 "Detalhes": "md=10/msl=3",
 "F1_Score": round(f1_score(decisionTree_predict_v4, y_test), 3),
 "Acurácia": round(accuracy_score(decisionTree_predict_v4, y_test), 3),
 "AUC": round(roc_auc_score(y_test, decisionTree_predict_v4), 3) #AUC inverte a ordem dos dados -  primeiro real depois previsões
}

with open("./models/modelo_v4.pkl", "wb") as file:  # "wb" significa escrita em modo binário
    pickle.dump(modelo_v2_decisionTree, file)


modelo_v4_metricas

{'Modelo': 'Decision Tree',
 'Versão': '4',
 'Detalhes': 'md=10/msl=3',
 'F1_Score': 0.751,
 'Acurácia': 0.72,
 'AUC': 0.72}

### <font color="red">Modelo v5 => SVC

In [78]:
clf_svc = SVC()

param = {
 'kernel' : ['Linear', 'rbf', 'poly']
}

modelo_v7_SVC_GS = GridSearchCV(clf_svc, param, cv = 3) # cv

modelo_v7_SVC_GS.fit(x_train, y_train)


### <font color="red">Modelo v6 => Adaboost

In [38]:
clf_adaboost = AdaBoostClassifier()

estimador_base = DecisionTreeClassifier(criterion = 'gini',
                                        max_depth = 10,
                                        min_samples_leaf = 1, 
                                        min_samples_split = 3)

modelo_v6_adaBoost = AdaBoostClassifier(base_estimator = estimador_base,
                                        learning_rate = 0.5,
                                        n_estimators = 300,
                                        algorithm='SAMME')


modelo_v6_adaBoost.fit(x_train, y_train)

modelo_v6_adaBoost.get_params()


{'algorithm': 'SAMME',
 'base_estimator__ccp_alpha': 0.0,
 'base_estimator__class_weight': None,
 'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': 10,
 'base_estimator__max_features': None,
 'base_estimator__max_leaf_nodes': None,
 'base_estimator__min_impurity_decrease': 0.0,
 'base_estimator__min_samples_leaf': 1,
 'base_estimator__min_samples_split': 3,
 'base_estimator__min_weight_fraction_leaf': 0.0,
 'base_estimator__random_state': None,
 'base_estimator__splitter': 'best',
 'base_estimator': DecisionTreeClassifier(max_depth=10, min_samples_split=3),
 'estimator': None,
 'learning_rate': 0.5,
 'n_estimators': 300,
 'random_state': None}

In [41]:
adaboost_predict_v5 = modelo_v6_adaBoost.predict(x_test)

In [43]:

modelo_v5_metricas = {
 "Modelo": "Adaboost",
 "Versão": "5",
 "Detalhes": "Decision Tree",
 "F1_Score": round(f1_score(adaboost_predict_v5, y_test), 3),
 "Acurácia": round(accuracy_score(adaboost_predict_v5, y_test), 3),
 "AUC": round(roc_auc_score(y_test, adaboost_predict_v5), 3) #AUC inverte a ordem dos dados -  primeiro real depois previsões
}

with open("./app/models/modelo_v5.pkl", "wb") as file:  # "wb" significa escrita em modo binário
    pickle.dump(modelo_v6_adaBoost, file)

modelo_v5_metricas

{'Modelo': 'Adaboost',
 'Versão': '5',
 'Detalhes': 'Decision Tree',
 'F1_Score': 0.855,
 'Acurácia': 0.855,
 'AUC': 0.855}

In [50]:
resumo = pd.DataFrame({"Modelo_V1": pd.Series(modelo_v1_metricas),
                       "Modelo_V2": pd.Series(modelo_v2_metricas),
                       "Modelo_V3": pd.Series(modelo_v3_metricas),
                       "Modelo_V4": pd.Series(modelo_v4_metricas),
                       "Modelo_V5": pd.Series(modelo_v5_metricas)})

In [51]:
resumo

,Modelo_V1,Modelo_V2,Modelo_V3,Modelo_V4,Modelo_V5
Modelo,Logistic Regression,Logistic Regression GS,Multinomial Naive Bayes,Decision Tree,Adaboost
Versão,1,2,3,4,5
Detalhes,-,C=09,alfa=1,md=10/msl=3,Decision Tree
F1_Score,0.882,0.888,0.856,0.751,0.855
Acurácia,0.881,0.887,0.859,0.72,0.855
AUC,0.881,0.887,0.859,0.72,0.855


## Decisão final

Utilização do Modelo_v2 - Logistic Regression Grid Search

In [52]:
!pip show scikit-learn


Name: scikit-learn
Version: 1.2.1
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: c:\users\alyss\anaconda3\lib\site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: imbalanced-learn, scikit-learn-intelex, scikit-plot


In [52]:
!pip freeze > requirements.txt
